<a href="https://colab.research.google.com/github/gulabpatel/Speech-to-Text/blob/main/03_VideoCaptionGenerationDemo_HF_wav2vec2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

video walkthrough the code : https://www.youtube.com/watch?v=7KaEC5LOEmc

In [1]:
!pip install -q pytube pydub transformers

In [2]:
from pytube import YouTube 
from pydub import AudioSegment
import librosa
import soundfile as sf

In [3]:
audio_path = 'C://'
link='https://www.youtube.com/watch?v=dT_aIJZ7Mxw'

try: 
    yt = YouTube(link) 
except: 
    print("Connection Error")

yt.streams.filter(file_extension='mp4')

[<Stream: itag="18" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="22" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="137" mime_type="video/mp4" res="1080p" fps="30fps" vcodec="avc1.640028" progressive="False" type="video">, <Stream: itag="136" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001f" progressive="False" type="video">, <Stream: itag="135" mime_type="video/mp4" res="480p" fps="30fps" vcodec="avc1.4d401f" progressive="False" type="video">, <Stream: itag="134" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.4d401e" progressive="False" type="video">, <Stream: itag="133" mime_type="video/mp4" res="240p" fps="30fps" vcodec="avc1.4d4015" progressive="False" type="video">, <Stream: itag="160" mime_type="video/mp4" res="144p" fps="30fps" vcodec="avc1.4d400c" progressive="False" type="video

In [4]:
## Download audio file
stream = yt.streams.get_by_itag(139)
stream.download(audio_path,"GoogleImagen.mp4")

'/content/C://GoogleImagen.mp4'

In [5]:
## convert mp4 to wav format
given_audio = AudioSegment.from_file('/content/C:/GoogleImagen.mp4', format="mp4")  
given_audio.export("GoogleImagen.wav", format="wav")

<_io.BufferedRandom name='GoogleImagen.wav'>

In [6]:
## using soundfile lib convert 480kps to 16000 kps as it is required for our model
sp, rate = sf.read("GoogleImagen.wav")
sp= librosa.resample(sp.T, rate, 16000)
## write back again with the same name
sf.write("GoogleImagen.wav", sp.T, 16000, subtype='PCM_24')

In [7]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:752: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Below code from: https://github.com/huggingface/transformers/issues/10366

In [8]:
transcript = ""
# Ensure that the sample rate is 16k
print(librosa.get_samplerate("GoogleImagen.wav"))

# Stream over 30 seconds chunks rather than load the full file to avoid the out of memory error
stream = librosa.stream(
    "GoogleImagen.wav",
    block_length=5,
    frame_length=16000,
    hop_length=16000
)

16000


In [9]:
for speech in stream:
        if len(speech.shape) > 1:
            speech = speech[:, 0] + speech[:, 1]

        input_values = tokenizer(speech, return_tensors="pt").input_values
        logits = model(input_values).logits

        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = tokenizer.decode(predicted_ids[0])
        print(transcription)
        

IN THIS WIDIO LET'S LOOK AT IMEGEN A NEW EIR SYSTEM FROM GOUGLE BRAIN
RATE SO ORDERS IMAGINE IMAGINE TICKS ARE TEXED AS AN IMPURT AND INGIN
WRITS IMAGES FOR EXAMPLE THE TEXT OVER HERE IS A CRON PLATTER DU
WITH A GOLDEN BEAK ARGUING WITH AN ANGRY TURTLE IN A FOREST SO THIS THEIR TEXT GIVEN
AND THIS IS THE IMAGE EN RATED SO HOW DOES THIS SYSTEM WORK O QUET
SO THIS SYSTEM TAKES TEXT AS AN INPUT THARE IS A FROZEN TEX
IN QUARDER WHICH PRODUCES A TEXTAIM BIDDING THIS TEXTAM BIDDING IS THEN FED INTO A SE
RES OFF IMAGE DEFUSION MORALS O KIT THAT IS A FUST TAKES TOO IMAGE DEFICIENT MO
ODEL WE GENERATES ANY MITCH THEN THERE IS A SUPERIAL RESOLUTION THE FI
SH AND MORTAL WHICH IS USED FOR INCREASING THE RESOLUTION HOKE THEN YOU HAVE
ANOTHER SUPERRESOLUTION DEFICIENT MORTAL WHICH GIVES ZUDHA FINAL IMAGE AND EAT
O OF THESE DEFICIENT MORALS TEXTHE TEXTAM BRIDDING AS THEY PUT ALONG WITH THE PREV
HIS OUTPUT OF THE PREVIOUS STAGE HOKE SO FOR THER TEXT OR IN CORDEROVER HE
ER THEY USER T FAI EXEXL EN CORD

In [10]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:752: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
for speech in stream:
        if len(speech.shape) > 1:
            speech = speech[:, 0] + speech[:, 1]

        input_values = tokenizer(speech, return_tensors="pt").input_values
        logits = model(input_values).logits

        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = tokenizer.decode(predicted_ids[0])
        print(transcription)